# 🎓 **Maestría en Inteligencia Artificial Aplicada**



[![Institution](https://img.shields.io/badge/Institution-Tecnológico%20de%20Monterrey-1F497D?style=for-the-badge&logo=graduation-cap)](https://tec.mx)




---

## **Proyecto Integrador**

### **Profesores**
- **Profesor Titular:** Dra. Grettel Barceló Alonso
- **Director Nacional de MNA:** Dr. Luis Eduardo Falcón Morales
- **Profra. Asistente:** Mtra. Verónica Sandra Guzmán de Valle

### **Patrocinadores**
- **Líder del Proyecto IMSS:** Dra. Ruth Pérez-Hernández
- **Investigadora en Psiquiatría:** Dra. Lina Díaz Castro:

---

## **Equipo #1**

### 🚀 **Nuestro Equipo**

<table style="width:100%; border:none;">
<tr>
<td style="text-align:center; border:none; padding:20px;">

### **Javier Augusto Rebull Saucedo**
<img src="https://iili.io/KuvsGKx.png" alt="Javier Augusto Rebull Saucedo" width="180">

**Matrícula:** `A01795838`  
🎓 MNA Student
</td>
<td style="text-align:center; border:none; padding:20px;">

### **Juan Carlos Pérez Nava**
<img src="https://iili.io/KuvP4gp.png" alt="Juan Carlos Pérez Nava" width="180">

**Matrícula:** `A01795941`  
🎓 MNA Student

</td>
<td style="text-align:center; border:none; padding:20px;">

### **Luis Gerardo Sánchez Salazar**
<img src="https://iili.io/KuvLaJp.png" alt="Luis Gerardo Sánchez Salazar" width="180">

**Matrícula:** `A01232963`  
🎓 MNA Student
</td>
</tr>
</table>

# **Antecedentes**

Los datos utilizados en este proyecto provienen del **Boletín Epidemiológico** publicado por la **Dirección General de Epidemiología** en México. Este boletín se actualiza de forma **semanal** a través del **Sistema Nacional de Vigilancia Epidemiológica (SINAVE)**, y contiene información oficial sobre el comportamiento de distintas enfermedades a nivel nacional y por entidad federativa.

La fuente oficial de descarga y consulta es el portal del Gobierno de México:  
https://www.gob.mx/salud/acciones-y-programas/direccion-general-de-epidemiologia-boletin-epidemiologico

Este proyecto fue realizado académicamente en colaboración del **Tecnológico de Monterrey** y el **IMSS**, como parte del curso de **Proyecto Integrador** de la **Maestría en Inteligencia Artificial Aplicada (MNA)**, durante el periodo **Enero–Abril 2026**.

---

# **Introducción**

El Boletín Epidemiológico se publica en formato PDF y, aunque es una fuente confiable, presenta retos técnicos importantes para su análisis automatizado: variaciones de formato entre años, tablas con estructura no uniforme, y diferencias en cómo aparecen los números y encabezados.

Por esta razón, este notebook implementa un flujo reproducible para **descargar, leer, limpiar y transformar** la información contenida en los PDFs, con el fin de construir un dataset consolidado que pueda ser utilizado en etapas posteriores de análisis estadístico y modelado predictivo.

---

# **Objetivo del Notebook**

El objetivo principal de este notebook es desarrollar un pipeline que permita:

- **Extraer automáticamente** la información relevante desde los PDFs del Boletín Epidemiológico  
- **Estandarizar el formato** de las tablas para evitar inconsistencias entre años  
- **Transformar los datos** a un dataset estructurado y escalable (formato long)  
- **Habilitar una GUI** que permita actualizar el dataset fácilmente conforme se publique nueva información semanal, manteniéndolo lo más actualizado posible

Este enfoque busca que el proceso sea **repetible, mantenible y extensible**, facilitando futuras etapas de limpieza avanzada y el desarrollo de modelos de **series de tiempo** para pronóstico.

---

# **Descarga de PDFs desde Google Drive (Colab)**

**Objetivo:** bajar todos los archivos de una carpeta de Drive a `/content/pdf`.

**Entrada:**
- `FOLDER_URL`: link de la carpeta de Google Drive

**Proceso:**
- Instala `gdown`
- Crea `/content/pdf`
- Descarga el folder completo al directorio local

**Salida:**
- PDFs disponibles en: `/content/pdf`

In [2]:
import os

# Crear carpeta local (ajusta la ruta según tu proyecto)
os.makedirs("data/raw/pdf", exist_ok=True)

FOLDER_URL = "https://drive.google.com/drive/folders/18XmOTIIiJUghy_YriJtcww2dggjRjYpT?usp=drive_link"

# En notebook de VS Code usa ! o %
!gdown --folder "{FOLDER_URL}" -O data/raw/pdf

Retrieving folder contents
Processing file 1jh8Z3xb-Dc-_wmmsXXfb031p2g0NGqea 2014_sem01.pdf
Processing file 1VepXMOXr8lMOEtB96wwhlDSuDaq9hMCj 2014_sem05.pdf
Processing file 1BSpsrzU-4vG_BsLXhLz96GT__6yu3Ui5 2014_sem31.pdf
Processing file 1WfDucs94gQkeuJOaPscdpgBjZrACV8gy 2015_sem46.pdf
Processing file 1ggPHOQiaqNJQ1Q0be93kNpmpoY0tvCWf 2016_sem50.pdf
Processing file 141buDnwwNaeRYVqeIpcQYI_rgCuxWRCb 2017_sem23.pdf
Processing file 1RMD_mjg9fxkvb7dJo9HFxpPrq7lZhFBU 2025_sem01.pdf
Processing file 1aLhuJYlhxchfMWtLUwTJN2N2i6xH-Nx2 2025_sem02.pdf
Processing file 1YKr3gEExyCvFwb-SjUEXB88sTvIDRvu8 2025_sem48.pdf
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1jh8Z3xb-Dc-_wmmsXXfb031p2g0NGqea
To: /Users/haowei/Documents/Integrador/EpiForecast-MX/notebooks/data/raw/pdf/2014_sem01.pdf
100%|████████████████████████████████████████| 560k/560k [00:00<00:00, 6.62MB/s]
Downloading...
From: htt

# **Listado de PDFs descargados**

**Objetivo:** detectar y listar todos los archivos `.pdf` dentro de `/content/pdf`.

**Entrada:**
- Carpeta local: `/content/pdf`

**Proceso:**
- Filtra solo archivos que terminen en `.pdf` (case-insensitive)
- Ordena alfabéticamente para mantener consistencia

**Salida:**
- `pdf`: lista con nombres de archivos PDF
- Imprime el total y el listado completo

In [4]:
pdf = sorted([f for f in os.listdir("data/raw/pdf") if f.lower().endswith(".pdf")])
print(f"Found {len(pdf)} PDFs")
print("\n".join(pdf))

Found 9 PDFs
2014_sem01.pdf
2014_sem05.pdf
2014_sem31.pdf
2015_sem46.pdf
2016_sem50.pdf
2017_sem23.pdf
2025_sem01.pdf
2025_sem02.pdf
2025_sem48.pdf


# **Instalación + imports (Camelot + PyPDF)**

**Objetivo:** preparar el entorno para leer PDFs y extraer tablas.

**Entrada:**
- Runtime de Colab (Linux)

**Proceso:**
- Instala `ghostscript` (requerido por Camelot)
- Instala `camelot-py` y `pypdf`
- Importa módulos para regex, manejo de dataframes y lectura/escritura de PDF

**Salida:**
- `camelot` listo para extraer tablas
- `PdfReader/PdfWriter` listos para leer y generar PDFs

In [5]:
import re
import pandas as pd
import camelot
from pypdf import PdfReader, PdfWriter

# **Detección de página + extracción de Año/Semana**

**Objetivo:** encontrar la página "buena" del PDF (donde vienen las 3 enfermedades) y extraer `Año` y `Semana`.

**Entrada:**
- `pdf_path`: ruta del PDF
- `KEYWORDS`: lista de palabras clave obligatorias (todas deben aparecer)

**Proceso:**
- Lee el PDF con `PdfReader`
- Recorre cada página y extrae su texto
- Valida que existan todas las keywords en la misma página
- Aplica 2 regex posibles para capturar `Semana` y `Año`:
  - `SEMANA_REGEX`: detecta formato tipo **"Semana 12 ... 2024"**
  - `SEMANA_REGEX_2`: detecta formato tipo **"semana epidemiológica 42 del 2024"**
- Si hay keywords pero no hay match, devuelve `8888` y `99` como bandera de error

**Salida:**
- `(page, year, week)` si se encuentra
- `(None, None, None)` si no se encuentra ninguna página válida

In [6]:
KEYWORDS = ["Depresión", "Parkinson", "Alzheimer"]  # deben aparecer juntas en la misma página

SEMANA_REGEX = re.compile(r"Semana\s+(\d{1,2}).*?(\d{4})", re.IGNORECASE)  # "Semana 12 ... 2024"
SEMANA_REGEX_2 = re.compile(r"semana\s+epidemiol[oó]gica\s+(\d{1,2})\s+del\s+(\d{4})", re.IGNORECASE)  # "semana epidemiológica 42 del 2024"

def find_page_and_week(pdf_path, KEYWORDS):
    reader = PdfReader(pdf_path)  # abre el PDF
    for i, page in enumerate(reader.pages):  # recorre páginas
        text = page.extract_text() or ""  # extrae texto (si falla, string vacío)
        if all(k.lower() in text.lower() for k in KEYWORDS):  # valida presencia de TODAS las keywords
            m = SEMANA_REGEX.search(text)  # busca patrón tipo "Semana 12 ... 2024"
            if m: week, year = m.groups(); return i + 1, int(year), int(week)  # i+1 = página humana (1-based)
            m2 = SEMANA_REGEX_2.search(text)  # busca patrón tipo "semana epidemiológica 42 del 2024"
            if m2: week2, year2 = m2.groups(); return i + 1, int(year2), int(week2) + 1  # +1 por tu ajuste
            return i + 1, 8888, 99  # encontró keywords pero no pudo extraer semana/año
    return None, None, None  # no encontró página con las keywords

# **Detección masiva de página + Año/Semana (por PDF)**

**Objetivo:** correr `find_page_and_week()` sobre todos los PDFs descargados y guardar el resultado en una tabla.

**Entrada:**
- `pdf`: lista de nombres de PDFs en `/content/pdf`
- `KEYWORDS`: palabras clave obligatorias
- `find_page_and_week(pdf_path, KEYWORDS)`

**Proceso:**
- Recorre cada archivo PDF
- Detecta `(página, año, semana)` en la página que contiene las keywords
- Guarda todo en `results` como tuplas
- Convierte a `df_results` para visualizarlo fácil

**Salida:**
- `results`: lista de tuplas `(Archivo, Página, Año, Semana)`
- `df_results`: DataFrame con columnas legibles para validar rápido

In [7]:
PDF_DIR = "data/raw/pdf"

results = []
for f in pdf:
    page, year, week = find_page_and_week(f"{PDF_DIR}/{f}", KEYWORDS)
    results.append((f, page, year, week))

df_results = pd.DataFrame(results, columns=["Archivo", "Página detectada", "Año", "Semana"])
df_results  # en VS Code notebooks esto muestra la tabla

,Archivo,Página detectada,Año,Semana
0,2014_sem01.pdf,3,2014,1
1,2014_sem05.pdf,58,2014,5
2,2014_sem31.pdf,60,2014,31
3,2015_sem46.pdf,60,2015,46
4,2016_sem50.pdf,60,2016,50
5,2017_sem23.pdf,60,2017,23
6,2025_sem01.pdf,72,2025,1
7,2025_sem02.pdf,70,2025,2
8,2025_sem48.pdf,76,2025,48


# **Extracción de tablas con Camelot + resumen de tamaños**

**Objetivo:** extraer tablas de la página detectada en cada PDF y verificar rápidamente su tamaño.

**Entrada:**
- `results`: tuplas `(Archivo, Página, Año, Semana)`
- `camelot.read_pdf(..., flavor="stream")`

**Proceso:**
- Crea `all_tables` como diccionario `{archivo: tables}`
- Salta PDFs sin página válida (`page is None`)
- Extrae tablas solo de la página detectada (no todo el PDF)
- Calcula el tamaño `(filas, columnas)` de `tables[0].df` para validar consistencia

**Salida:**
- `all_tables`: tablas extraídas por archivo
- `df_sizes`: resumen por PDF con `Año/Semana/Página` y dimensiones de la tabla

In [9]:
PDF_DIR = "data/raw/pdf"

print("Tablas encontradas:")
all_tables = {}
for f, page, year, week in results:
    if page is None:
        continue
    all_tables[f] = camelot.read_pdf(f"{PDF_DIR}/{f}", pages=str(page), flavor="stream")

print(all_tables)
print("\n\n")

print("Tamaño de las tablas")
rows = []
for f, tables in all_tables.items():
    page, year, week = next((p, y, w) for (ff, p, y, w) in results if ff == f)
    nrows, ncols = tables[0].df.shape
    rows.append([f, year, week, page, nrows, ncols])

df_sizes = pd.DataFrame(rows, columns=["Archivo", "Año", "Semana", "Página", "Filas", "Columnas"])
print(df_sizes.to_string(index=False))

Tablas encontradas:
{'2014_sem01.pdf': <TableList n=2>, '2014_sem05.pdf': <TableList n=1>, '2014_sem31.pdf': <TableList n=1>, '2015_sem46.pdf': <TableList n=1>, '2016_sem50.pdf': <TableList n=1>, '2017_sem23.pdf': <TableList n=1>, '2025_sem01.pdf': <TableList n=1>, '2025_sem02.pdf': <TableList n=1>, '2025_sem48.pdf': <TableList n=1>}



Tamaño de las tablas
       Archivo  Año  Semana  Página  Filas  Columnas
2014_sem01.pdf 2014       1       3     28         2
2014_sem05.pdf 2014       5      58     39        12
2014_sem31.pdf 2014      31      60     37        10
2015_sem46.pdf 2015      46      60     41        13
2016_sem50.pdf 2016      50      60     39        13
2017_sem23.pdf 2017      23      60     39        13
2025_sem01.pdf 2025       1      72     38        13
2025_sem02.pdf 2025       2      70     36        18
2025_sem48.pdf 2025      48      76     38        13


# **Impresión de tablas extraídas (por PDF y por tabla)**

**Objetivo:** visualizar todas las tablas detectadas por Camelot en cada PDF, con contexto (archivo, año, semana, página).

**Entrada:**
- `all_tables`: diccionario `{archivo: tables}`
- `results`: lista con `(Archivo, Página, Año, Semana)`

**Proceso:**
- Recorre cada archivo en `all_tables`
- Recupera `(page, year, week)` desde `results`
- Imprime cada tabla `t.df` con un separador visual y un encabezado claro

**Salida:**
- Tablas completas impresas en consola, agrupadas por archivo y numeradas (`Tabla 1`, `Tabla 2`, etc.)

In [10]:
for f, tables in all_tables.items():
    page, year, week = next((p, y, w) for (ff, p, y, w) in results if ff == f)  # metadata del PDF
    for i, t in enumerate(tables):  # recorre todas las tablas detectadas en esa página y las imprime
        print("*" * 100)
        print(f"Archivo: {f} | Año: {year} | Semana: {week} | Página: {page}")
        print(f"Tabla {i+1}")
        print("*" * 100)
        print(t.df.to_string(index=False))


****************************************************************************************************
Archivo: 2014_sem01.pdf | Año: 2014 | Semana: 1 | Página: 3
Tabla 1
****************************************************************************************************
                                                                            0                                     1
                                  Vigilancia Epidemiológica Semana  01,  2014                                     3
                                                                   puerperio*                                      
captura dentro de\nla plataforma\nelectró-\n10. Efectos del calor y de la luz                                      
         nica del SUIVE por\nlas 19 846 unidades\n11. Enfermedad de Alzheimer                                      
       médicas incorporadas a este sistema y que\n12. Enfermedad de Parkinson                                      
                                  

# **¿Qué pasa con la información extraída?**

La extracción desde PDFs no produce datos "limpios" automáticamente. Por esto, se deben realizar algunas etapas de preprocesamiento a los datos. Cada paso del pipeline propuesto se implementa para corregir variaciones reales del Boletín y mantener la información lo más fiel posible al PDF antes de entrar a limpieza avanzada y modelado.

Primero tuvimos que **encontrar la página correcta y la semana/año**. Algunos PDFs incluyen el texto con formatos distintos, y en ciertos casos el contenido puede venir como imagen o el texto extraído es incompleto. Por eso usamos **dos expresiones regulares (regex)** como alternativas: una para patrones tipo "Semana 12 … 2024" y otra para "semana epidemiológica 42 del 2024". Esto aumenta la probabilidad de detectar correctamente `(Año, Semana)` sin depender de un único formato.

Después, al extraer tablas con Camelot, vimos que el resultado trae **ruido estructural**. Por ejemplo, aparecen **columnas completamente vacías** (separadores o espacios) que no existen como datos en el PDF. Por eso aplicamos una limpieza que **elimina columnas 100% vacías** dentro del rango real de estados (Aguascalientes…Zacatecas). Esto ayuda a **normalizar el ancho de la tabla** y evitar desalineaciones al mapear columnas.

Luego, aplicamos `clean_df()` para quedarnos solo con filas que realmente son "Estado + números". Los PDFs suelen incluir encabezados, notas o totales ("ENTIDAD", "TOTAL", "FUENTE", "NOTA", etc.) que Camelot puede capturar como filas normales. Además, para identificar filas válidas, contamos celdas que parecen numéricas, pero sin forzar conversiones todavía.

En el contenido numérico encontramos varios formatos:
- **Millares separados por espacios** (`"1 450"`) o por **comas** (`"1,450"`) → ambos deben interpretarse como `1450`.
- Valores `"-"` o `""` que en realidad representan **cero** en el boletín.
- Valores como `"n.e."` (no existente) u otros textos, que deben tratarse como **faltantes reales** (`NA`).

Por eso usamos `normalize_number()` para estandarizar cada celda a tres salidas consistentes: `int`, `0`, o `NA`. Esto evita "inventar datos" y mantiene trazabilidad para análisis posterior.

También detectamos que **el formato varía por año**. Algunos PDFs **no traen la columna de "Acumulado año anterior"**. Para no tener esquemas distintos por año, usamos `pad_prev_year_cols()` para **rellenar esa columna faltante con `NA`**. Así todos los años quedan con el mismo formato (mismas columnas por padecimiento), y el reshaping funciona igual siempre.

Finalmente, aplicamos dos reshapes con objetivos distintos:
- **Wide**: 1 fila por entidad, con columnas por padecimiento y métrica. Es ideal para **comparar visualmente** contra la tabla original del PDF y validar que la extracción quedó alineada.
- **Long**: 1 fila por (Entidad + Padecimiento + Semana). Este es el formato correcto para construir el **dataset consolidado histórico**, hacer agregaciones, validaciones y preparar modelos.

Aún falta procesamiento: en esta etapa nos enfocamos en **extraer y estandarizar lo más pegado al PDF**. Después viene una limpieza más exhaustiva (validación de rangos, consistencia temporal, outliers, faltantes, etc.) para finalmente pasar a análisis y modelos de series de tiempo.

A continuación, pasaremos a explicar los pasos del pipeline utilizados.

# **Limpieza: eliminar columnas completamente vacías (solo dentro del bloque de estados)**

**Objetivo:** quitar columnas "basura" que Camelot a veces crea (separadores/espacios), pero **solo** si están vacías en el rango real de datos.

**Entrada:**
- `df`: DataFrame extraído por Camelot
- `start_state`: estado donde inicia el bloque (default: `Aguascalientes`)
- `end_state`: estado donde termina el bloque (default: `Zacatecas`)

**Proceso:**
- Renombra columnas a índices numéricos para trabajar estable
- Localiza las filas donde aparecen `start_state` y `end_state`
- Recorta el DF al rango de filas "real" (solo estados)
- Elimina columnas que sean **100% vacías** dentro de ese rango

**Salida:**
- DataFrame con columnas vacías removidas (sin tocar columnas que sí tienen datos)

In [11]:
def eliminar_columnas_vacias(df, start_state="Aguascalientes", end_state="Zacatecas"):
    """Elimina columnas 100% vacías dentro del bloque de estados (Aguascalientes..Zacatecas)."""
    df = df.copy()
    df.columns = range(df.shape[1])  # fuerza columnas numéricas (0..N-1) para evitar nombres raros de Camelot

    col0 = df[0].astype(str).str.strip()  # columna de estados normalizada

    try:
        i_start = col0[col0.eq(start_state)].index[0]  # fila donde empieza el bloque de estados
        i_end = col0[col0.eq(end_state)].index[0]      # fila donde termina el bloque de estados
    except IndexError:
        return df  # si no encuentra el rango, no modifica nada

    if i_start > i_end: i_start, i_end = i_end, i_start  # asegura orden correcto

    sub = df.loc[i_start:i_end, :]  # analiza solo filas del bloque real (evita encabezados/pies)

    is_blank = sub.astype(str).apply(lambda col: col.str.strip().eq(""))  # True donde la celda está vacía
    keep_cols = is_blank.mean(axis=0) < 1.0  # conserva columnas que NO son 100% vacías en el bloque

    return df.loc[:, keep_cols]  # devuelve DF sin columnas inútiles


# **Limpieza de tabla Camelot: quedarte con "Estado + números"**

**Objetivo:** filtrar la tabla extraída para conservar únicamente filas que parecen datos reales por estado, eliminando encabezados, notas y ruido.

**Entrada:**
- `df`: DataFrame crudo (`tables[0].df`) de Camelot
- `min_numeric_cells`: mínimo de celdas "numéricas" requeridas en columnas 1..N (default: `2`)

**Proceso:**
- Elimina columnas 100% vacías dentro del bloque Aguascalientes..Zacatecas
- Normaliza nombres de estados (columna 0) y elimina filas vacías
- Descarta filas tipo encabezado/pie: `ENTIDAD`, `TOTAL`, `FUENTE`, `NOTA`, etc.
- Valida filas contando celdas "numéricas" en columnas 1..N:
  - Enteros detectados tras quitar separadores de miles (espacios/comas)
  - `"-"` y `""` cuentan como 0 (se tratan como válidas para la estructura)
  - Conserva filas con `numeric_count >= min_numeric_cells`

**Salida:**
- DataFrame limpio, reindexado, listo para `pad_prev_year_cols()` y `reshape()`

In [12]:
def clean_df(df, min_numeric_cells=2):
    """Deja solo filas tipo 'Estado + datos' y elimina ruido típico de Camelot."""
    df = eliminar_columnas_vacias(df)  # quita columnas 100% vacías dentro del bloque de estados

    df.columns = range(df.shape[1]); df[0] = df[0].astype(str).str.strip()  # índices estables + estados sin espacios
    df = df[df[0].ne("")]  # fuera filas sin estado
    df = df[~df[0].str.match(r"^(ENTIDAD|FEDERATIVA|TOTAL.*|FUENTE.*|NOTA.*)$", case=False, na=False)]  # fuera headers/footers

    num_cols = [c for c in df.columns if c != 0]  # columnas numéricas (todo menos estado)
    cells = df[num_cols].astype(str).apply(lambda col: col.str.strip())  # normaliza whitespace para evaluar
    cells_clean = cells.replace(r"[ ,]", "", regex=True)  # "1 450"/"1,450"->"1450" solo para conteo

    is_zeroish = cells.apply(lambda col: col.eq("-") | col.eq(""))  # "-" o "" cuentan como 0 válido
    is_int = cells_clean.apply(lambda col: col.str.fullmatch(r"\d+").fillna(False))  # detecta enteros puros
    numeric_count = (is_int | is_zeroish).sum(axis=1)  # cuántas celdas "numéricas" tiene cada fila

    df = df[numeric_count >= min_numeric_cells]  # conserva filas que realmente parecen datos (no texto suelto)
    return df.reset_index(drop=True)  # reindex final limpio

# **Estandarización de columnas: agregar "Año anterior" cuando falta**

**Objetivo:** forzar un mismo esquema de columnas aunque algunos PDFs no traigan la columna de comparación con año anterior.

**Entrada:**
- `df`: tabla ya limpia (por ejemplo salida de `clean_df`)
- `keywords`: lista de padecimientos (k = cantidad)

**Lógica clave:**
- Esquema esperado:
  - Con año anterior: `1 + 4*k` columnas
  - Sin año anterior: `1 + 3*k` columnas
- Si `df` tiene exactamente `1 + 3*k`, entonces por cada padecimiento:
  - Conserva `total`, `hombres`, `mujeres`
  - Agrega `total_prev = pd.NA` como columna faltante

**Salida:**
- DataFrame con `1 + 4*k` columnas, listo para `reshape()` sin condicionales por año

In [13]:
def pad_prev_year_cols(df: pd.DataFrame, keywords: list[str]) -> pd.DataFrame:
    """Si falta la col 'año anterior' (3 cols por padecimiento), inserta esa col como NA (4 cols por padecimiento)."""
    df = df.copy()
    k = len(keywords); no_prev = 1 + 3 * k; with_prev = 1 + 4 * k  # 1=Entidad; luego 3k o 4k métricas

    if df.shape[1] != no_prev: return df  # solo actúa cuando el DF está exactamente en esquema sin año anterior

    out = {0: df[0]}  # mantiene Entidad en col 0
    for i, kw in enumerate(keywords):
        old_base = 1 + i * 3; new_base = 1 + i * 4  # mapea bloques de 3->4 columnas por padecimiento
        out[new_base + 0] = df[old_base + 0]  # total semana
        out[new_base + 1] = df[old_base + 1]  # acumulado hombres
        out[new_base + 2] = df[old_base + 2]  # acumulado mujeres
        out[new_base + 3] = pd.NA             # acumulado año anterior (no existe en el DF original)

    return pd.DataFrame(out)  # devuelve DF con esquema “completo” (con año anterior)

# **Normalización de números (celdas Camelot → int/0/NA)**

**Objetivo:** convertir valores de tabla a números confiables sin romper casos raros.

**Entrada:**
- `x`: valor crudo de una celda (string, número, vacío, guion, etc.)

**Reglas:**
- `NA` real → `pd.NA`
- `""` o `"-"` → `0`
- Elimina separadores de miles (`" "` y `","`)
- Si queda un entero (`^\d+$`) → `int`
- Cualquier otro formato → `pd.NA`

**Salida:**
- `int` cuando es válido
- `0` cuando representa "sin dato numérico"
- `pd.NA` cuando el contenido no es interpretable como entero

In [14]:
def normalize_number(x):
    if pd.isna(x): return pd.NA  # mantiene NA reales

    s = str(x).strip()
    if s == "" or s == "-": return 0  # vacíos/guiones se tratan como cero

    s2 = s.replace(" ", "").replace(",", "")  # quita separadores de miles: "1 450"/"1,450"->"1450"
    if re.fullmatch(r"\d+", s2): return int(s2)  # acepta solo enteros puros

    return pd.NA  # cualquier valor raro (texto, símbolos, etc.)

# **Mapeo de columnas por padecimiento (para reshape automático)**

**Objetivo:** construir un diccionario que diga qué columnas pertenecen a cada padecimiento (sin hardcodear índices).

**Entrada:**
- `keywords`: lista de padecimientos (en el orden que aparecen en la tabla)
- `start_col`: primera columna numérica (default `1`, porque `0` es Entidad)
- `step`: tamaño del bloque por padecimiento (default `4` = total, hombres, mujeres, año anterior)

**Proceso:**
- Para cada padecimiento calcula su `base`
- Asigna índices relativos:
  - `total = base`
  - `hombres = base + 1`
  - `mujeres = base + 2`
  - `total_prev = base + 3`

**Salida:**
- `col_map`: diccionario listo para usar en `reshape()` y `reshape_wide()`

In [15]:
def build_column_map(keywords: list[str], start_col: int = 1, step: int = 4):
    col_map = {}  # {padecimiento: {total,hombres,mujeres,total_prev}}
    for i, disease in enumerate(keywords):
        base = start_col + i * step  # inicio del bloque de columnas de ese padecimiento
        col_map[disease] = {"total": base, "hombres": base + 1, "mujeres": base + 2, "total_prev": base + 3}
    return col_map  # índices de columnas para reshape()

# **Reshape "wide" (1 fila por Entidad, columnas por padecimiento)**

**Objetivo:** convertir la tabla de Camelot a un formato "ancho", buscando el formato original del boletín

**Entrada:**
- `df`: tabla limpia y estandarizada (idealmente después de `clean_df` + `pad_prev_year_cols`)
- `year`, `week`: año y semana detectados del PDF
- `col_map`: mapeo de columnas por padecimiento (`build_column_map`)
- `normalize_number`: normaliza valores a `int/0/NA`

**Proceso:**
- Crea 1 registro por fila del DF (Entidad)
- Agrega metadatos: `Anio`, `Semana`, `Entidad`
- Para cada padecimiento agrega 4 métricas como columnas nuevas:
  - `Casos_semana_{kw}`
  - `Acumulado_hombres_{kw}`
  - `Acumulado_mujeres_{kw}`
  - `Acumulado_anio_anterior_{kw}`

**Salida:**
- DataFrame "wide": 1 fila por Entidad, muchas columnas (métrica × padecimiento)

In [16]:
def reshape_wide(df: pd.DataFrame, year: int, week: int, col_map: dict) -> pd.DataFrame:
    """Salida “ancha”: 1 fila por entidad; columnas separadas por padecimiento y métrica."""
    records = []
    for _, row in df.iterrows():
        rec = {"Anio": year, "Semana": f"{week:02d}", "Entidad": row[0]}  # metadata fija por fila
        for kw, cols in col_map.items():  # kw=padecimiento, cols=índices de columnas
            rec[f"Casos_semana_{kw}"] = normalize_number(row[cols["total"]])  # total semanal
            rec[f"Acumulado_hombres_{kw}"] = normalize_number(row[cols["hombres"]])  # acumulado H
            rec[f"Acumulado_mujeres_{kw}"] = normalize_number(row[cols["mujeres"]])  # acumulado M
            rec[f"Acumulado_anio_anterior_{kw}"] = normalize_number(row[cols["total_prev"]])  # acumulado año anterior
        records.append(rec)
    return pd.DataFrame(records)  # DF ancho: útil para guardar CSV por semana/página

# **Reshape "long" (1 fila por Entidad + Padecimiento)**

**Objetivo:** transformar la tabla en formato "largo" para consolidar todo el histórico en un solo dataset.

**Entrada:**
- `df`: tabla limpia + con columnas completas (ideal después de `pad_prev_year_cols`)
- `year`, `week`: metadatos del PDF
- `col_map`: índices de columnas por padecimiento
- `normalize_number`: convierte valores a `int/0/NA`

**Proceso:**
- Recorre cada Entidad (fila)
- Por cada Entidad genera una fila por cada padecimiento en `col_map`
- Cada fila final contiene:
  - `Anio`, `Semana`, `Entidad`, `Padecimiento`
  - `Casos_semana`, `Acumulado_hombres`, `Acumulado_mujeres`, `Acumulado_anio_anterior`

**Salida:**
- DataFrame "long": muchas filas, pocas columnas (perfecto para unir semanas/años y modelado)

In [17]:
def reshape(df: pd.DataFrame, year: int, week: int, col_map: dict) -> pd.DataFrame:
    records = []
    for _, row in df.iterrows():
        estado = row[0]  # entidad/estado
        for disease, cols in col_map.items():  # 1 fila por (estado, padecimiento)
            records.append({
                "Anio": year, "Semana": f"{week:02d}", "Entidad": estado, "Padecimiento": disease,
                "Casos_semana": normalize_number(row[cols["total"]]),
                "Acumulado_hombres": normalize_number(row[cols["hombres"]]),
                "Acumulado_mujeres": normalize_number(row[cols["mujeres"]]),
                "Acumulado_anio_anterior": normalize_number(row[cols["total_prev"]]),
            })
    return pd.DataFrame(records)  # DF largo: ideal para análisis y consolidado histórico

# **Procesamiento por archivo (RAW → CLEAN → PAD → WIDE) + resumen indexable**

**Objetivo:** procesar todos los PDFs ya cargados en `all_tables`, guardar los DataFrames intermedios y generar un resumen para navegar por `idx`.

**Entrada:**
- `all_tables`: `{archivo: tables}` (Camelot ya ejecutado)
- `results`: `(Archivo, Página, Año, Semana)`
- `KEYWORDS`, `col_map`

**Proceso (por archivo):**
- `RAW`: `tables[0].df`
- `CLEAN`: `clean_df(RAW)`
- `PAD`: `pad_prev_year_cols(CLEAN, KEYWORDS)`
- `WIDE`: `reshape_wide(PAD, year, week, col_map)`
- Guarda todo en `processed` junto con metadata (archivo/año/semana/página)

**Salida:**
- `processed`: lista de dicts con `RAW/CLEAN/PAD/WIDE` por archivo (para inspección por índice)
- `df_summary`: tabla con `idx` + `shape` por etapa para detectar outliers rápido

In [18]:
print(f"Col_map:")
col_map = build_column_map(KEYWORDS)  # índices por padecimiento (total/h/m/prev)
display(col_map)

Col_map:


{'Depresión': {'total': 1, 'hombres': 2, 'mujeres': 3, 'total_prev': 4},
 'Parkinson': {'total': 5, 'hombres': 6, 'mujeres': 7, 'total_prev': 8},
 'Alzheimer': {'total': 9, 'hombres': 10, 'mujeres': 11, 'total_prev': 12}}

In [19]:
processed = []  # lista de dicts con metadata + dfs por etapa
for f, tables in all_tables.items():
    page, year, week = next((p, y, w) for (ff, p, y, w) in results if ff == f)  # usa results como “source of truth”
    if tables.n == 0: continue  # skip si Camelot no detectó nada

    df_raw = tables[0].df  # tabla cruda
    df_clean = clean_df(df_raw)  # limpia ruido/filtra filas útiles
    df_pad = pad_prev_year_cols(df_clean, KEYWORDS)  # asegura col de año anterior si falta
    df_wide = reshape_wide(df_pad, year, week, col_map)  # 1 fila por entidad, columnas por padecimiento

    processed.append({"Archivo": f, "Año": year, "Semana": week, "Página": page, "RAW": df_raw, "CLEAN": df_clean, "PAD": df_pad, "WIDE": df_wide})

# tabla ligera para inspección rápida
df_summary = pd.DataFrame([{
    "idx": i,
    "Archivo": x["Archivo"],
    "Año": x["Año"],
    "Semana": x["Semana"],
    "Página": x["Página"],
    "RAW_shape": x["RAW"].shape,
    "CLEAN_shape": x["CLEAN"].shape,
    "PAD_shape": x["PAD"].shape,
    "WIDE_shape": x["WIDE"].shape,
} for i, x in enumerate(processed)])
display(df_summary)


,idx,Archivo,Año,Semana,Página,RAW_shape,CLEAN_shape,PAD_shape,WIDE_shape
0,0,2014_sem01.pdf,2014,1,3,"(28, 2)","(0, 2)","(0, 2)","(0, 0)"
1,1,2014_sem05.pdf,2014,5,58,"(39, 12)","(32, 10)","(32, 13)","(32, 15)"
2,2,2014_sem31.pdf,2014,31,60,"(37, 10)","(32, 10)","(32, 13)","(32, 15)"
3,3,2015_sem46.pdf,2015,46,60,"(41, 13)","(32, 13)","(32, 13)","(32, 15)"
4,4,2016_sem50.pdf,2016,50,60,"(39, 13)","(32, 13)","(32, 13)","(32, 15)"
5,5,2017_sem23.pdf,2017,23,60,"(39, 13)","(32, 13)","(32, 13)","(32, 15)"
6,6,2025_sem01.pdf,2025,1,72,"(38, 13)","(32, 13)","(32, 13)","(32, 15)"
7,7,2025_sem02.pdf,2025,2,70,"(36, 18)","(32, 13)","(32, 13)","(32, 15)"
8,8,2025_sem48.pdf,2025,48,76,"(38, 13)","(32, 13)","(32, 13)","(32, 15)"


# **Inspector por índice (ver todas las etapas de un archivo)**

**Objetivo:** seleccionar un archivo procesado por índice y visualizar cómo cambia el DataFrame en cada etapa.

**Entrada:**
- `n`: índice dentro de `processed`
- `processed[n]`: dict con metadata + `RAW/CLEAN/PAD/WIDE`

**Proceso:**
- Imprime metadata: archivo, año, semana, página
- Muestra cada etapa con su `shape`:
  - `RAW` (Camelot)
  - `CLEAN` (filtrado)
  - `PAD_PREV` (columnas completas)
  - `WIDE` (formato ancho)

**Salida:**
- Visualización completa del pipeline para ese archivo específico

In [20]:
n = 1  # <--  índice del archivo a inspeccionar

x = processed[n]  # dict con metadata + dfs por etapa
print(f'INDEX: {n} | {x["Archivo"]} | Año: {x["Año"]} | Semana: {x["Semana"]:02d} | Página: {x["Página"]}')  # header

print("\nRAW:", x["RAW"].shape); display(x["RAW"])       # tabla cruda de Camelot
print("\nCLEAN:", x["CLEAN"].shape); display(x["CLEAN"]) # tabla limpia
print("\nPAD_PREV:", x["PAD"].shape); display(x["PAD"])  # columnas estandarizadas
print("\nWIDE:", x["WIDE"].shape); display(x["WIDE"])    # reshape ancho final

INDEX: 1 | 2014_sem05.pdf | Año: 2014 | Semana: 05 | Página: 58

RAW: (39, 12)


,0,1,2,3,4,5,6,7,8,9,10,11
0,FEDERATIVA,,,,,,,,,,,
1,,,2014,,,2014,,,2014,,,
2,,,Acum.,,,Acum.,,,,Acum.,,
3,,Sem.,,,Sem.,,,Sem.,,,,
4,,,M,F,,M,F,,M,,F,
5,Aguascalientes,1,2,3,-,-,1,-,,-,,-
6,Baja California,45,30,121,6,7,8,3,,1,,9
7,Baja California Sur,13,5,18,2,2,3,-,,-,,-
8,Campeche,10,6,27,2,1,2,1,,1,,1
9,Coahuila,2,8,42,-,8,4,-,,-,,-



CLEAN: (32, 10)


,0,1,2,3,4,5,6,7,8,9
0,Aguascalientes,1,2,3,-,-,1,-,-,-
1,Baja California,45,30,121,6,7,8,3,1,9
2,Baja California Sur,13,5,18,2,2,3,-,-,-
3,Campeche,10,6,27,2,1,2,1,1,1
4,Coahuila,2,8,42,-,8,4,-,-,-
5,Colima,17,11,41,-,2,2,2,-,2
6,Chiapas,20,12,31,6,6,4,1,1,1
7,Chihuahua,43,25,116,8,10,7,2,1,5
8,Distrito Federal,212,124,329,10,11,16,4,3,2
9,Durango,9,6,30,-,1,2,-,-,-



PAD_PREV: (32, 13)


,0,1,2,3,4,5,6,7,8,9,10,11,12
0,Aguascalientes,1,2,3,<NA>,-,-,1,<NA>,-,-,-,<NA>
1,Baja California,45,30,121,<NA>,6,7,8,<NA>,3,1,9,<NA>
2,Baja California Sur,13,5,18,<NA>,2,2,3,<NA>,-,-,-,<NA>
3,Campeche,10,6,27,<NA>,2,1,2,<NA>,1,1,1,<NA>
4,Coahuila,2,8,42,<NA>,-,8,4,<NA>,-,-,-,<NA>
5,Colima,17,11,41,<NA>,-,2,2,<NA>,2,-,2,<NA>
6,Chiapas,20,12,31,<NA>,6,6,4,<NA>,1,1,1,<NA>
7,Chihuahua,43,25,116,<NA>,8,10,7,<NA>,2,1,5,<NA>
8,Distrito Federal,212,124,329,<NA>,10,11,16,<NA>,4,3,2,<NA>
9,Durango,9,6,30,<NA>,-,1,2,<NA>,-,-,-,<NA>



WIDE: (32, 15)


,Anio,Semana,Entidad,Casos_semana_Depresión,Acumulado_hombres_Depresión,Acumulado_mujeres_Depresión,Acumulado_anio_anterior_Depresión,Casos_semana_Parkinson,Acumulado_hombres_Parkinson,Acumulado_mujeres_Parkinson,Acumulado_anio_anterior_Parkinson,Casos_semana_Alzheimer,Acumulado_hombres_Alzheimer,Acumulado_mujeres_Alzheimer,Acumulado_anio_anterior_Alzheimer
0,2014,05,Aguascalientes,1,2,3,<NA>,0,0,1,<NA>,0,0,0,<NA>
1,2014,05,Baja California,45,30,121,<NA>,6,7,8,<NA>,3,1,9,<NA>
2,2014,05,Baja California Sur,13,5,18,<NA>,2,2,3,<NA>,0,0,0,<NA>
3,2014,05,Campeche,10,6,27,<NA>,2,1,2,<NA>,1,1,1,<NA>
4,2014,05,Coahuila,2,8,42,<NA>,0,8,4,<NA>,0,0,0,<NA>
5,2014,05,Colima,17,11,41,<NA>,0,2,2,<NA>,2,0,2,<NA>
6,2014,05,Chiapas,20,12,31,<NA>,6,6,4,<NA>,1,1,1,<NA>
7,2014,05,Chihuahua,43,25,116,<NA>,8,10,7,<NA>,2,1,5,<NA>
8,2014,05,Distrito Federal,212,124,329,<NA>,10,11,16,<NA>,4,3,2,<NA>
9,2014,05,Durango,9,6,30,<NA>,0,1,2,<NA>,0,0,0,<NA>


# **Consolidación final (reshape "long" + concat de todos los archivos)**

**Objetivo:** convertir cada tabla procesada a formato "long" y unir todo en un solo DataFrame.

**Entrada:**
- `processed`: lista con dicts que ya incluyen `PAD`, `Año`, `Semana`, `Archivo`
- `col_map`: mapeo de columnas por padecimiento
- `reshape(df, year, week, col_map)`

**Proceso:**
- Para cada elemento en `processed`:
  - Aplica `reshape()` sobre `x["PAD"]`
  - Agrega columna `Archivo` para rastrear origen
- Concatena todo con `pd.concat(...)`

**Salida:**
- `df_all_long`: dataset consolidado (1 fila por Entidad + Padecimiento + Semana)

In [21]:
all_long = []  # aquí juntamos el DF "long" de cada archivo
for x in processed:
    df_long = reshape(x["PAD"], x["Año"], x["Semana"], col_map)  # PAD ya está limpio+estandarizado
    df_long["Archivo"] = x["Archivo"]  # trazabilidad
    all_long.append(df_long)

df_all_long = pd.concat(all_long, ignore_index=True)  # consolidado final
display(df_all_long)


/var/folders/0p/fpm4_3l94jxbbc__pnl93bmw0000gn/T/ipykernel_17515/1274023842.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_all_long = pd.concat(all_long, ignore_index=True)  # consolidado final


,Archivo,Anio,Semana,Entidad,Padecimiento,Casos_semana,Acumulado_hombres,Acumulado_mujeres,Acumulado_anio_anterior
0,2014_sem05.pdf,2014.0,05,Aguascalientes,Depresión,1,2.0,3.0,NaN
1,2014_sem05.pdf,2014.0,05,Aguascalientes,Parkinson,0,0.0,1.0,NaN
2,2014_sem05.pdf,2014.0,05,Aguascalientes,Alzheimer,0,0.0,0.0,NaN
3,2014_sem05.pdf,2014.0,05,Baja California,Depresión,45,30.0,121.0,NaN
4,2014_sem05.pdf,2014.0,05,Baja California,Parkinson,6,7.0,8.0,NaN
...,...,...,...,...,...,...,...,...,...
763,2025_sem48.pdf,2025.0,48,Yucatán,Parkinson,2,97.0,72.0,139.0
764,2025_sem48.pdf,2025.0,48,Yucatán,Alzheimer,1,7.0,9.0,30.0
765,2025_sem48.pdf,2025.0,48,Zacatecas,Depresión,52,519.0,1820.0,2647.0
766,2025_sem48.pdf,2025.0,48,Zacatecas,Parkinson,1,19.0,22.0,22.0


# **Conclusiones**

Este notebook permitió construir una base sólida para transformar los PDFs del Boletín Epidemiológico en un dataset estructurado y reutilizable. A lo largo del proceso se confirmó que, aunque la fuente es oficial y se publica semanalmente, el formato no es completamente uniforme entre años, por lo que es indispensable aplicar reglas de estandarización antes de cualquier análisis.

Se automatizó la detección de la página relevante y la extracción de `Año` y `Semana` mediante estrategias robustas (regex alternativas), lo cual es clave para mantener trazabilidad temporal y permitir actualizaciones continuas. También se normalizaron valores numéricos con distintos formatos (millares con comas o espacios, guiones que representan ceros y valores no existentes), evitando errores silenciosos y asegurando consistencia en el dataset final.

Adicionalmente, se resolvió el problema de tablas incompletas en ciertos años (sin columna de "Acumulado año anterior") mediante relleno controlado con `NA`, garantizando un esquema estándar que permite consolidación y comparación entre periodos. Finalmente, el reshaping en formato **wide** y **long** permitió tanto validar visualmente contra el PDF como preparar un dataset adecuado para análisis longitudinal y modelado predictivo.

Es importante destacar que este notebook representa los **cimientos técnicos** sobre los cuales está construida nuestra **GUI**, ya que concentra la lógica esencial de extracción, limpieza y transformación. La GUI fue desarrollada pensando en un proyecto más formal y mantenible, no limitado a Google Colab, sino diseñado para vivir dentro de un **repositorio en GitHub**, con estructura ordenada, versionamiento, documentación y capacidad de actualización continua conforme se publique nueva información.

En esta etapa el objetivo fue extraer la información lo más apegada posible al boletín original. Aún queda trabajo por realizar en limpieza avanzada y validaciones de consistencia, pero el pipeline ya deja listo el camino para iniciar análisis más profundos y, posteriormente, modelos de series de tiempo con datos actualizados semana a semana.

# **Key Highlights**

- Este notebook establece los **cimientos del pipeline** (PDF → tabla → dataset) sobre los que se construyó la GUI.
- Se logró detectar **página correcta + Año/Semana** de forma robusta usando **2 regex alternativos**.
- Se normalizaron inconsistencias típicas del boletín: **millares con comas/espacios**, `"-"` como **0**, y `"n.e."` como **NA**.
- Se estandarizó el esquema entre años rellenando la columna faltante de **"Acumulado año anterior"** con `NA`.
- Se generaron dos vistas: **wide** para validar contra el PDF y **long** para consolidación y análisis.
- La solución quedó lista para **actualización semanal** conforme se publique nueva información.
- El diseño está pensado para vivir en un **repo en GitHub**, mantenible y ordenado, como parte del entregable.